In [3]:
import requests
import time
import random

import pandas as pd

from bs4 import BeautifulSoup

In [4]:
#Scraper 1 (getting links to publications)

StartDate = '01/01/2000'
EndDate = '09/22/2022'

url = 'https://www.nist.gov/publications/search?k=&t=&a=&ps=All&n=&d%5Bmin%5D=' + StartDate + '&d%5Bmax%5D=' + EndDate

text = requests.get(url).text
soup = BeautifulSoup(text, 'html.parser')

nPages = int(str(soup.find_all('footer')[0])[36:-14]) // 25 + 1

d = []
t = []
l = []
for page in range(nPages):
    time.sleep(random.uniform(0.1, 2))
    url = 'https://www.nist.gov/publications/search?k=&t=&a=&ps=All&n=&d%5Bmin%5D=' + StartDate + '&d%5Bmax%5D=' + EndDate + '&page=' + str(page)
    text = requests.get(url).text
    soup = BeautifulSoup(text, 'html.parser')
    dates = soup.find_all('div', {"class": "nist-teaser__date"})
    for date in dates:
        s = str(date)[40:-13]
        d.append(s)
    titles = soup.find_all('h3', {"class": "nist-teaser__title"})
    for title in titles:
        s = str(title)[34:-9]
        s = s[s.find('>') + 1:]
        t.append(s)
        s = str(title)[40:] 
        s = 'https://www.nist.gov' + s[:s.find('>') - 1]
        l.append(s)

df = pd.DataFrame()
df['titles'] = t
df['links'] = l
df['dates'] = d
df.dates = pd.to_datetime(df.dates)   

In [5]:
#Scraper 2 (getting publication details)
df[['authors', 'abstracts', 'fields', 'keywords']] = ''

for i in range(len(df)):
    time.sleep(random.uniform(0.1, 0.3))
    url = df.links[i]
    text = requests.get(url).text
    soup = BeautifulSoup(text, 'html.parser')
    names = soup.find_all('div', {"class": "nist-field__item"})
    s = str(names[1]).replace('<span class="nist-author">', '').replace('</span>', '')[30:-6].replace(',   ', ', ').replace('  ', ' ')
    df.loc[i, 'authors'] = s
    abstract = soup.find_all('div', {"class": "text-with-summary"})
    df.loc[i, 'abstracts'] = str(abstract)[59:-10]
    if soup.find_all('div', {"class": "nist-tags"}):
        fields = soup.find_all('div', {"class": "nist-tags"})[0].find_all('a')
        f = ''
        for field in fields:
            f += field.text
            f += ', '
        df.loc[i, 'fields'] = f[:-2]
    keywords = soup.find_all('div', {"class": "nist-field__item"})
    df.loc[i, 'keywords'] = keywords[-1].text.replace(',  ', ', ')

In [8]:
df

,titles,links,dates,authors,abstracts,fields,keywords
0,NIST Micronutrients Measurement Quality Assura...,https://www.nist.gov/publications/nist-micronu...,2022-09-21,"David L. Duewer, Jeanice ""Brown Thomas "", Marg...","From 1984 to 2017, the National Institute of S...",Chemistry,"Apparent Precision, Carotenoids, Coenzyme Q₁₀,..."
1,Certification of Standard Reference Material(R...,https://www.nist.gov/publications/certificatio...,2022-09-20,"Thomas W. Vetter, Jason Waters, Adam L. Pintar",Standard Reference Material (SRM) 915c Calcium...,"Standards, Health, Chemistry","Calcium, Calcium Carbonate, Carbonate, Coulome..."
2,Near-edge x-ray absorption and emission of cya...,https://www.nist.gov/publications/near-edge-x-...,2022-09-20,"Karina Bzheumikhova, John Vinson, Yves Kayser,...",The comparability and reliability of the analy...,Condensed matter,Local Download
3,Profile of the IoT Core Baseline for Consumer ...,https://www.nist.gov/publications/profile-iot-...,2022-09-20,"Katerina N. Megas, Michael Fagan, Jeffrey Marr...",This publication documents the consumer profil...,"Securing emerging technologies, Internet of Th...","Internet of Things (IoT), consumer IoT, cybers..."
4,Using a Single-Zone Residential Model to Evalu...,https://www.nist.gov/publications/using-single...,2022-09-20,"Stephen Zimmerman, Brian Polidoro, Lisa Ng, Wi...","On March 11, 2020, the World Health Organizati...",Indoor air quality,Local Download
...,...,...,...,...,...,...,...
39813,Waveguide atom beam splitter for laser-cooled ...,https://www.nist.gov/publications/waveguide-at...,2000-01-01,"D Mueller, Eric A. Cornell, M Prevedelli, Pete...",,,Journals
39814,Wear Analysis of UHMWPE Using a load Sum Method,https://www.nist.gov/publications/wear-analysi...,2000-01-01,"M C. Shen, Stephen M. Hsu, John A. Tesk, A Chr...",,,"UHMWPE, biomaterials, crystalline polymers, lo..."
39815,Well-ionization Chamber Response Relative to N...,https://www.nist.gov/publications/well-ionizat...,2000-01-01,"Michael G. Mitch, Brian E. Zimmerman, P J. Lam...",,,Journals
39816,X-ray and extreme ultraviolet emission induced...,https://www.nist.gov/publications/x-ray-and-ex...,2000-01-01,"E Parra, I Alexeev, Jingyun Fan, Y K. Kim, S J...",,,Journals


In [7]:
df.to_csv('NIST_publications_2000-2022.csv')